# Laboratorio 1 - Informe

### Grupo 4:
     - S. Calvo C.I 5.711.417-7     
     - X. Iribarnegaray C.I
     - J. Simonelli C.I 5.405.358-4

## 1. Objetivo

El objetivo de este laboratorio es:
- Implementar el algoritmo ID3, añadiendo el hiperparámetro *max_range_split*, que determina la cantidad máxima de rangos en los que se puede partir un atributo númerico.
- Utilizar scikit-learn para el preprocesamiento de datos y la creación de modelos basados en árboles de decisión.
- Evaluar y comparar los modelos generados.

## 2. Diseño

### 2.1 Preprocesamiento de datos
Previo a la ejecución del algoritmo se es provisto a este, mediante una variable global, el conjunto de feature values continuos. Luego, durante la ejecución del algoritmo, se discretizaran los valores asociados a estos feature values.


### 2.2 Algoritmo
En primer lugar, definimos la función `entropy(dataset, target)` que será utilizada a lo largo de la implementación del algoritmo ID3:


In [ ]:
import numpy as np

def entropy(dataset, target):
    # value_counts() returns a Series containing the counts of unique values
    values = dataset[target].value_counts()
    # shape returns the size of dataset, shape[0] being the number of rows
    total = dataset.shape[0]
    p0 = values.iloc[0]/total
    if (len(values) > 1):
        p1 = values.iloc[1]/total
        return -(p0)*np.log2(p0) - (p1) * np.log2(p1)
    else: 
        return -(p0)*np.log2(p0)

Luego, el siguiente paso a delinear en el algoritmo es la obtención del "mejor atributo". Esto lo logramos mediante la función\
`best_feature(dataset, target, features, continuous_features, max_range_splits)`\
que retorna el siguiente mejor atributo a elegir. Esta decisión es hecha en base a la fórmula de Ganancia vista en el curso, donde, para maximizar la ganancia, basta con minimizar el segundo factor de la fórmula, y, por lo tanto, se eligirá el atributo que minimice este valor.


In [ ]:
def best_feature(dataset, target, features, continuous_features, max_range_splits):
    conditional_entropies = []
    continuous = {}
    for feature in features:
        if feature in continuous_features:
            # Continuous-Valued feature 
            aux_entropy, best_split = get_splits(dataset, feature, target, max_range_splits)
            conditional_entropies.append(aux_entropy)
            continuous[feature] = best_split
        else :
            # Discrete-Valued feature 
            res = 0
            for value, count in dataset[feature].value_counts().items():
                res += count*entropy(dataset.loc[dataset[feature] == value], target)
            conditional_entropies.append(res / dataset.shape[0])
    best_feature = features[conditional_entropies.index(min(conditional_entropies))]
    
    if not (best_feature in continuous):
        return best_feature, None
    return best_feature, continuous[best_feature]


Para el caso en el que el atributo a evaluar sea discreto (`else` del `if`), solamente se lleva a cabo el cálculo descrito previamente.\
Por otro lado, para el caso de evaluar un atributo de valores continuos, se debe realizar otro procedimiento.


Dado que por defecto el algoritmo ID3 solamente aplica a ejemplares de valores discretos, en el caso de querer entrenar un modelo mediante un dataset que contiene valores continuos, será necesario discretizarlos en tiempo de ejecución.\
Esto es logrado mediante la división de los valores continuos, con valores calculados con el promedio de dos puntos con valores objetivo distintos. Luego, dependiendo del valor del hiperparámetro *max_range_splits* (de valor 2 o 3 ), utilizamos combinaciones de estos puntos para particionar los valores continuos en rangos. De esta manera, obtenemos finalmente un atributo de valores discretos.\
\
Sin embargo se debe tener en cuenta que, en datasets de gran tamaño, es probable que exista una gran cantidad de posibles puntos de corte, por lo cual es necesario identificar el mejor o mejor par de puntos. De manera similar al cálculo del "mejor atributo", se realiza un cálculo de la ganancia para cada split, y se elige el que minimice el segundo factor de la fórmula. Para esto, se utiliza la función 
`get_splits(dataset, feature, target, max_range_splits)`

In [ ]:
def get_splits(dataset, feature, target, max_range_splits):
    min_conditional_entropy = 2
    dataset = dataset.sort_values(by=feature)
    current_target = dataset[target].iloc[0]
    dataset_size = dataset.shape[0]
    candidate_splits = []
    best_splits = []
    
    # Finding splits
    for i in range(1, dataset_size):
        if current_target != dataset[target].iloc[i]:
            candidate_splits.append((dataset[feature].iloc[i-1] + dataset[feature].iloc[i])/2)
            current_target = dataset[target].iloc[i]
    
    sample = candidate_splits
    if len(candidate_splits) > 50:
        sample = random.sample(candidate_splits, 50)
    
    splits = generate_combinations(sample, max_range_splits)
 
    for split in splits:
        splitted_dataset = split_dataset(dataset, feature, split)
        aux_conditional_entropy = 0
        for value, count in splitted_dataset[feature].value_counts().items():
            aux_conditional_entropy += count*entropy(splitted_dataset.loc[splitted_dataset[feature] == value], target)
        aux_conditional_entropy = aux_conditional_entropy / splitted_dataset.shape[0]
            
        if (aux_conditional_entropy < min_conditional_entropy):
            min_conditional_entropy = aux_conditional_entropy
            best_splits = split
            
    return (min_conditional_entropy, best_splits)

Por su parte, la función `generate_combinations` se encarga de generar todas las posibles combinaciones de puntos de corte para un atributo continuo, y hemos decidido que, en caso de obtener más de 500 combinaciones, se elija un subconjunto aleatorio de tamaño 500 de estas, para evitar un tiempo de ejecución excesivo.\
Además, para alterar el dataset en tiempo de ejecución, se utiliza la función `split_dataset`, que se encarga de discretizar los valores de un atributo continuo en base a el o los puntos de corte obtenidos.\
A través de `get_splits`, obtenemos ya sea el mejor valor o mejor par de valores por los cuales dividir el atributo continuo, junto con el componente de entropía que se utilizará en `best_feature` para determinar el mejor atributo a elegir.

Finalmente, con todas estas funciones implementadas, obtenemos la siguiente implementación del algoritmo ID3:

In [ ]:
def id3(dataset, target, features, continuous_features, max_range_splits, intact_dataset):
    if len(features) == 0 or len(dataset[target].value_counts().index) == 1:
        # value_counts[0] is either the only or the most common target value left in the current dataset.
        return dataset[target].value_counts().index[0] 
 
    best, best_splits = best_feature(dataset, target, features, continuous_features, max_range_splits)
    decision_tree = {best: {}}
    
    new_features = features.copy()
    new_features.remove(best)
    
    original_dataset = intact_dataset
    
    if best_splits:
        original_dataset = split_dataset(intact_dataset, best, best_splits)
        dataset = split_dataset(dataset, best, best_splits)
        
    for value in original_dataset[best].value_counts().index:
        examples = dataset.loc[dataset[best] == value]
        if (len(examples) == 0):
            decision_tree[best][value] = original_dataset.loc[original_dataset[best] == value][target].value_counts().index[0]
        else:
            decision_tree[best][value] = id3(examples, target, new_features, continuous_features, max_range_splits, intact_dataset)
    
    return decision_tree

Este algoritmo comienza analizando si quedan features a analizar o si todos los ejemplos tienen el mismo valor en el dataset restante. En estos casos se etiquetan con el valor más común y se retorna esta etiqueta. En el segundo caso en particular, el valor más común siendo el único restante. Siguiendo el algoritmo ID3, calculamos el mejor atributo mediante la función `best_feature`, explicada anteriormente, y guardamos los mejores splits si el atributo es continuo. Una vez elegido el atributo, se crea un diccionario donde guardar las ramas del árbol que se generarán. Si el valor es discreto se itera por cada valor posible, usando los valores del conjunto de datos inicial, creando ramas por cada uno. En estas ramas, si no hay ejemplos de este valor, se asigna el valor más común de los ejemplos con este valor del conjunto de datos original y, de lo contrario, se llama de forma recursiva a la función id3 quitando este atributo como posibles atributos y usando como conjunto de datos los ejemplos del valor. Si es continuo, se itera sobre los valores posibles dada la discretización por `split_dataset` y, en el caso de que no haya ejemplos, se utiliza el valor más común del dataset_original utilizando la misma discretización.

## Comparativa entre preprocesamiento y max_range_splits = 2 con diferentes ratios train/test

In [ ]:
from id3 import get_splits, split_dataset, id3, split_into_train_test, test_instances, init
from datetime import datetime
import random

dataset, features, continuous_features, target = init()

for i in range(50,100,10):
       print('\n',i,'% entrenamiento, ',100-i, '% test' )
       train_ds, test_ds = split_into_train_test(dataset,i/100)
       
       # Preprocessed
       preprocessed_dataset = train_ds.copy()
       for cont_feature in continuous_features:
              entropy, splits = get_splits(preprocessed_dataset,cont_feature,target,2)
              preprocessed_dataset = split_dataset(preprocessed_dataset,cont_feature,splits)
       startTime = datetime.now()
       preprocessed_decision_tree = id3(preprocessed_dataset, target, features, [], 2, preprocessed_dataset)
       preprocessed_time = datetime.now() - startTime
       acierto_pre = test_instances(preprocessed_decision_tree,test_ds)
       print('Preprocessed: ', end='')
       print('Accuracy: ',acierto_pre,'%', ' Time: ', preprocessed_time)
       
       
       # Max_Range_Splits_2
       startTime = datetime.now()
       max_range_split_2_decision_tree = id3(train_ds,target,features, continuous_features, 2, train_ds)
       max_range_split_2_time = datetime.now() - startTime
       acierto_run = test_instances(max_range_split_2_decision_tree,test_ds)
       print('Max Range Split 2: ', end='')
       print('Accuracy: ',acierto_run,'%', ' Time: ', max_range_split_2_time)
       




## Comparativa entre preprocesamiento y max_range_splits = 3 con diferentes ratios train/test

In [ ]:
from id3 import get_splits, split_dataset, id3, split_into_train_test, test_instances, init
from datetime import datetime
import random

dataset, features, continuous_features, target = init()

random.seed(1)

for i in range(50,100,10):
       print('\n',i,'% entrenamiento, ',100-i, '% test' )
       train_ds, test_ds = split_into_train_test(dataset,i/100)
       
       # Preprocessed
       preprocessed_dataset = train_ds.copy()
       for cont_feature in continuous_features:
              entropy, splits = get_splits(preprocessed_dataset,cont_feature,target,2)
              preprocessed_dataset = split_dataset(preprocessed_dataset,cont_feature,splits)
       startTime = datetime.now()
       preprocessed_decision_tree = id3(preprocessed_dataset, target, features, [], 2, preprocessed_dataset)
       preprocessed_time = datetime.now() - startTime
       acierto_pre = test_instances(preprocessed_decision_tree,test_ds)
       print('Preprocessed: ', end='')
       print('Accuracy: ',acierto_pre,'%', ' Time: ', preprocessed_time)
       
       # Max_Range_Splits_3
       startTime = datetime.now()
       max_range_split_3_decision_tree = id3(train_ds,target,features, continuous_features, 3, train_ds)
       max_range_split_3_time = datetime.now() - startTime
       acierto_run = test_instances(max_range_split_3_decision_tree,test_ds)
       print('Max Range Split 3: ', end='')
       print('Accuracy: ',acierto_run,'%', ' Time: ', max_range_split_3_time)

## Random Forest vs Decision Tree Classifier vs Max_range_splits = 2 vs Max_range_splits = 3

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from id3 import init, split_into_train_test, id3, test_instances

dataset, features, continuous_features, target = init()

X = dataset.drop(target, axis=1)
y = dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

ohe = OneHotEncoder(sparse_output=False)

discrete_features = list(set(features) - set(continuous_features))

for feat in discrete_features:
    ohe.fit(dataset[feat].to_numpy().reshape(-1, 1))
    
    # Transform the training and test data using the fitted OneHotEncoder
    # Converts the categorical feature in X_train and X_test to one-hot encoded format
    new_train = ohe.transform(X_train[feat].to_numpy().reshape(-1,1))
    new_test = ohe.transform(X_test[feat].to_numpy().reshape(-1,1))
    
    # Create column names for the new one-hot encoded features
    column_names = [f"{feat}_{cat}" for cat in ohe.categories_[0]]
    
    for i, col_name in enumerate(column_names):
        # Add the new one-hot encoded columns to the X_train and X_test DataFrame
        X_train[col_name] = new_train[:, i]
        X_test[col_name] = new_test[:, i]

    

random_forest = RandomForestClassifier(random_state=0)
random_forest.fit(X_train, y_train)

decision_tree_classifier = DecisionTreeClassifier(random_state=0)
decision_tree_classifier.fit(X_train, y_train)

y_pred_random_forest = random_forest.predict(X_test)
accuracy_random_forest = accuracy_score(y_test, y_pred_random_forest)
print('Random forest accuracy: ', accuracy_random_forest*100, '%')

y_pred_decission_tree = decision_tree_classifier.predict(X_test)
accuracy_decision_tree_classifier = accuracy_score(y_test, y_pred_decission_tree)
print('Decission Tree Classifier accuracy: ', accuracy_decision_tree_classifier*100, '%')

train_ds, test_ds = split_into_train_test(dataset,0.8)

max_range_split_2_decision_tree = id3(train_ds,target,features, continuous_features, 2, train_ds)
acierto_split_2 = test_instances(max_range_split_2_decision_tree,test_ds)
print('Max Range Split 2 accuracy: ',acierto_split_2,'%')#, ' Time: ', preprocessed_time)

max_range_split_3_decision_tree = id3(train_ds,target,features, continuous_features, 3, train_ds)
acierto_split_3 = test_instances(max_range_split_3_decision_tree,test_ds)
print('Max Range Split 3 accuracy: ',acierto_split_3,'%')#, ' Time: ', preprocessed_time)




Random forest accuracy:  88.3177570093458 %
Decission Tree Classifier accuracy:  82.71028037383178 %
Max Range Split 2 accuracy:  83.17757009345794 %
Max Range Split 3 accuracy:  79.43925233644859 %


### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

dataset, features, continuous_features, target = init()

X = dataset.drop(target, axis=1)
y = dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

ohe = OneHotEncoder(sparse_output=False)

discrete_features = list(set(features) - set(continuous_features))

for feat in discrete_features:
    ohe.fit(dataset[feat].to_numpy().reshape(-1, 1))
    
    # Transform the training and test data using the fitted OneHotEncoder
    # Converts the categorical feature in X_train and X_test to one-hot encoded format
    new_train = ohe.transform(X_train[feat].to_numpy().reshape(-1,1))
    new_test = ohe.transform(X_test[feat].to_numpy().reshape(-1,1))
    
    # Create column names for the new one-hot encoded features
    column_names = [f"{feat}_{cat}" for cat in ohe.categories_[0]]
    
    for i, col_name in enumerate(column_names):
        # Add the new one-hot encoded columns to the X_train and X_test DataFrame
        X_train[col_name] = new_train[:, i]
        X_test[col_name] = new_test[:, i]

decision_tree_classifier = DecisionTreeClassifier(random_state=0)

decision_tree_classifier.fit(X_train, y_train)

y_pred = decision_tree_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print('Decision Tree Classifier accuracy: ', accuracy*100, '%')



### 2.3 Evaluación
Discutir:
    - Calculo de porcentaje
    - Division en subconjunto de entrenamiento y evaluacion- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.


Dado que por defecto algoritmo ID3 solamente aplica a ejemplares de valores discretos, en el caso de querer entrenar un modelo mediante un dataset que contiene valores continuos, será necesario discretizarlos en tiempo de ejecución. Esto es logrado mediante la división de los valores continuos utilizando *splits*, con posibles valores calculados con el promedio de dos puntos con valores objetivo distintos. Luego, dependiendo del valor del hiperparámetro *max_range_splits* (2 o 3 ), . Sin embargo, en datasets de gran tamaño, es probable que exista una gran cantidad de posibles splits, por lo cual es necesario identificar 

Cosas a mencionar:
    -Max split y discretizacion

## 2.3 Evaluación
Discutir:
    - Calculo de porcentaje
    - Division en subconjunto de entrenamiento y evaluacion- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.